# V1 --- Sagemaker Training Job for Base NER Model Fine tuning of BERT 
In this noteboook, I set up the training environment to run an AWS sagemaker fine tuning job of BERT for Named Entity Recognition (NER) with the CONLL2003 dataset. The resulting model will be used in the augmentation of the wikipedia toxic comments dataset into an NER dataset. I will also use the RAC (binary classifier of toxic statements) I trained to add a new named entity to the modified dataset as well. That being toxic/hateful entities. 

There is code for fine tuning BERT on the CONNL2003 in this notebook that was sourced from Mastering Transformers (chapter 6 starting at page 181) and was adapted to run in a sagemaker training job.
The github for that section of the book is found here https://github.com/PacktPublishing/Mastering-Transformers/blob/main/CH06/CH06a_Fine_tuning_language_models_for_NER.ipynb

In [2]:
#!pip install datasets

In [ ]:
import datasets
conll2003 = datasets.load_dataset('conll2003')

### A note about NER:
Named Entity Recognition is a token classification task where, for an input sequence, the objective is to determine which of the words/tokens belong to the different entities that abide within the sentence/text. As well as the added caveat of when those entity types first appear and when they are subsequently mentioned.

# Data Preperation

In [5]:
conll2003['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

#### Here are the NER tag names with their associated meaning

O: Outside of any named entity  
B-PER: Start of a person's name  
I-PER: Inside a person's name  
B-LOC: Start of a location  
I-LOC: Inside a location  
B-ORG: Start of an organization  
I-ORG: Inside an organization  
B-MISC: Start of a miscellaneous entity  
I-MISC: Inside a miscellaneous entity  

In [6]:
example_0_tags = conll2003['train'][0]['ner_tags'] # integer encoded tags for example zero
ner_tag_names = conll2003['train'].features["ner_tags"].feature.names #get tag names 

decoded_target = [ner_tag_names[i] for i in example_0_tags] # human readable tags
original_sentence = ' '.join(conll2003['train'][0]['tokens']) # get original sentence

print(f'For the sentence: {original_sentence}\nThe tags are: {decoded_target}') # print the results

For the sentence: EU rejects German call to boycott British lamb .
The tags are: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


#### The output above means that the organization 'EU' begins on the first token. Then the rest of the tokens are either not named entities or miscelaneous entities.

# Data Preparation

##### Here I am going to load in the tokenizer for use with distilbert. It is BertTokenizerFast

In [9]:
#!pip install transformers

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

##### Below I am tokenizing the sentence about the EU boycott from above. Notice that there are now 12 input_ids as opposed to the 8 original tokens. This is because the tokenizer I am using is a subword tokenizer.

In [11]:
#NER tasks like the conll2003 are whitespaced tokenized, so we must specify this to the tokenizer
tokenizer([original_sentence], is_split_into_words=True)

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

##### Because we are using a subword tokenizer, it will create discrepancies between the length of the original labels and the now longer, length of the input_ids sequence. The function below adresses this issue.

In [12]:
label_all_tokens = True # set special tokens to be included in the labels

def tokenize_and_align(examples):
    '''
    This functions job is to align the NER labels of text after tokenizing with a subword tokenizer.
    First, it tokenizes the text. Then it iterates through the labels of the text and, through a conditional
    statement, assigns subwords as well as special tokens the appropriate label. This is accoomplished by 
    referencing the word index of the subword token and comparing it to the previous word index. 

    this is necessary data prep for this task given the use of a subword tokenizer. 
    
    This function is to be mapped to the entire dataset
    '''

    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) # tokenize the sentence
    labels = [] # initialize a list to store the label ids

    for i, label in enumerate(examples["ner_tags"]): # iterate over the labels 
        word_ids = tokenized_inputs.word_ids(batch_index=i) # get the word ids for the current sentence
        previous_word_idx = None 
        label_ids = [] 
        for word_idx in word_ids: 
            if word_idx is None: # special tokens like [CLS] and [SEP] have a word id of None 
                label_ids.append(-100) # so we set the label id to -100 for the model
            elif word_idx != previous_word_idx: # if the word id is different from the previous one, it is a new word
                label_ids.append(label[word_idx])  # so we append the label id
            else: 
                label_ids.append(label[word_idx] if label_all_tokens else -100) #subword tokens of a single word will share the same word idx, so we set the label id to the previous label or -100 if we don't want to use the subword labels
            previous_word_idx = word_idx  # update the previous word id
        labels.append(label_ids) # append the list of label ids for the current sentence
    tokenized_inputs["labels"] = labels # reasign the labels with the added subword labels for the current sentence
    return tokenized_inputs

##### Here we map the tokenize_and_align function from above to the entire dataset. .map() is a datasets method different from python's built in function map()

In [ ]:
tokenized_dataset = conll2003.map(tokenize_and_align, batched=True)

##### This is the structure of the tokenized_dataset

In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

##### Below is a single example from the train set. The last four key value pairs are new additions.

In [15]:
tokenized_dataset["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [101,
  7270,
  22961,
  1528,
  1840,
  1106,
  21423,
  1418,
  2495,
  12913,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]}

##### Here I am saving the model locally,

In [ ]:
tokenized_dataset.save_to_disk('tokenized_conll2003')

##### This is because am going to save the model to s3 so that the hugging face estimator can access it during training.
##### Below I am creating a new bucket to store the data (and model) in.

In [17]:
import boto3

s3 = boto3.client('s3', region_name='us-east-2')
bucket_name = 'conll2003-task'

# Create a new bucket in the us-east-2 region
s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})

# Check the location of an existing bucket
response = s3.get_bucket_location(Bucket=bucket_name)
bucket_location = response['LocationConstraint']
print('Bucket location:', bucket_location)

Bucket location: us-east-2


##### Here I upload the data to the bucket just created

In [22]:
import os

def upload_directory_to_s3(directory_path, s3_bucket, s3_key_prefix=''):
    '''
    uploads datasets stored in folders to a new dir in s3 bucket 
    for estimator to access. dir names are that which
    they are pulled from.
    '''
    directory_name = os.path.basename(directory_path)
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            s3_key = os.path.join(s3_key_prefix, directory_name, os.path.relpath(file_path, directory_path))
            s3_bucket.Object(s3_key).upload_file(Filename=file_path)
            
#specify bucket for upload
s3 = boto3.resource('s3')
bucket_name = 'conll2003-task'
bucket = s3.Bucket(bucket_name)

#folder path to datset
folder_path = "/home/ec2-user/SageMaker/NER_training_sagemaker/tokenized_conll2003"

upload_directory_to_s3(folder_path, bucket)

##### Now that this data is uploaded it to s3 it will be accessed within the training docker

# Model Preperation

In [25]:
#!pip install transformers

##### I am downloading distilbert-base-uncased for this fine tuning job.

In [35]:
from transformers import AutoModelForTokenClassification

#there are 9 labels for the token classification task because there are 9 ner-tags
model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=9) 

##### We will also need a data collator. The job of the data collator is to convert the data into the correct format before being passed into the model. For example: giving padding to sequences shorter than the max length

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer) 

##### I am saving the model and tokenizer (from above) to the disk in order to upload to s3 and access within the training docker.

In [ ]:
model.save_pretrained("distilbert-base-uncased")
tokenizer.save_pretrained("bert-tokenizer-fast")

##### Below I upload them

In [32]:
folder_paths = [
    "/home/ec2-user/SageMaker/NER_training_sagemaker/bert-tokenizer-fast",
    "/home/ec2-user/SageMaker/NER_training_sagemaker/distilbert-base-uncased"
]

#upload both tokenizer and model
for folder_path in folder_paths:
    upload_directory_to_s3(folder_path, bucket)

# Overview of Training Metric

In [3]:
#!pip install seqeval

##### Within the training script, we must provide a compute_metrics() function to the Trainer object. Below is that function. It prepares model outputs so that precision, recall, f1, and accuracy can be computed with seqeval.

In [58]:
import numpy as np
metric = datasets.load_metric("seqeval") #load in seqeval metric
    

def compute_metrics(p): 
    '''
        this function unpacks the predictions and labels from p. Then it applies argmax to the prediction logics which converts
        them to indices within the labels_list. Then assigned to true_predictions is a list comprehension of those indices converted 
        to their label names. The true_labels list has this analogous operation performed on the label indices of the targets for 
        that example. Then the true_predictiosn and true_labels are evaluated for precision, recall, and f1 using the seqeval package.
    '''
    #NER labels specific to the conll2003 task
    label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'] 
        
    #unpack predictions
    predictions, labels = p 
        
    #get prediction indices for use in labels_list by argmaxing logits
    predictions = np.argmax(predictions, axis=2) 
    
    #prediction indicies ---> labels
    true_predictions = [ 
        [label_list[pred] for (pred, lab) in zip(prediction, label) if lab != -100] for prediction, label in zip(predictions, labels) 
    ] 
    
    #Ground truth indicies ---> labels
    true_labels = [ 
        [label_list[lab] for (pred, lab) in zip(prediction, label) if lab != -100] for prediction, label in zip(predictions, labels) 
    ] 
    
    #get score
    results = metric.compute(predictions=true_predictions, references=true_labels) 
    
    return { 
        "precision": results["overall_precision"], 
        "recall": results["overall_recall"], 
        "f1": results["overall_f1"], 
        "accuracy": results["overall_accuracy"], 
    } 

#### Here I am running an example forward pass to test the comput_metric() function

In [53]:
import torch

#get example from pre-processed dataset
tokenized_example = tokenized_dataset['train'][0]

# Convert lists to PyTorch tensors and add a batch dimension
input_ids = torch.tensor([tokenized_example['input_ids']])
#token_type_ids = torch.tensor([tokenized_example['token_type_ids']])
attention_mask = torch.tensor([tokenized_example['attention_mask']])

# Run a forward pass through the model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

# Extract the logits
logits = outputs.logits

# get labels 
labels = torch.tensor([tokenized_dataset['train'][0]['labels']])


##### The shapes output below suggest everything is in order. The labels are integer encoded. After argmaxing the logits, they will be the same. The two can then be evaluated in the compute_metrics function.

In [56]:
print(f'logits shape {logits.shape}')
print(f'lables shape {labels.shape}')

logits shape torch.Size([1, 12, 9])
lables shape torch.Size([1, 12])


##### Evaluate forward pass against labels

In [59]:
p = [logits, labels]
result = compute_metrics(p)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### This low performance is to be expected as we have not fine tuned the pretrained model yet

In [60]:
print(result)

{'precision': 0.1111111111111111, 'recall': 0.3333333333333333, 'f1': 0.16666666666666666, 'accuracy': 0.1}


# Sagemaker Env


In [1]:
import sagemaker
sess = sagemaker.Session() #this creates a sagemaker session -
role = sagemaker.get_execution_role() #this gets permissions from the env where 
                                      #it is running. I am running in a sagemaker notebook instance

# HuggingFace Estimator
##### The huggingface estimator is a tool that will create a docker image of our specified hyperparams and conduct the training specified within the train.py training script.

In [2]:
from sagemaker.huggingface import HuggingFace


# hyperparameters which are passed to the training job
hyperparameters={'epochs': 3,
                 'per_device_train_batch_size': 16,
                 'per_device_eval_batch_size': 16#,
                 #'model_name_or_path': 's3://conll2003-task/distilbert-base-uncased/',
                 #'tokenizer_name_or_path': 's3://conll2003-task/bert-tokenizer-fast/'
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='/home/ec2-user/SageMaker/NER_training_sagemaker', #change this to training script location
        instance_type='ml.g4dn.xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.28.1',
        pytorch_version='2.0.0',
        py_version='py310',
        hyperparameters = hyperparameters
)

In [4]:
huggingface_estimator.fit({
    'train': "s3://conll2003-task/tokenized_conll2003/train/", 
    'test': "s3://conll2003-task/tokenized_conll2003/validation/"
    })

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-08-28-22-06-28-271


2023-08-28 22:06:46 Starting - Starting the training job...
2023-08-28 22:07:02 Starting - Preparing the instances for training......
2023-08-28 22:07:54 Downloading - Downloading input data...
2023-08-28 22:08:19 Training - Downloading the training image................................................
2023-08-28 22:16:37 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-08-28 22:16:49,156 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-08-28 22:16:49,175 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-08-28 22:16:49,184 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-08-28 22:16:49,190 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-08-28 22:16:54,456 sagemaker-training-toolkit INF

Preparing metadata (setup.py): finished with status 'done'
Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=c7e64cb2a1f2c84fdf7bef3c05bd303ca74695a29e1532622cbccab7bfd0fcbd
Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
/opt/ml/code/train.py:65: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval") #load in seqeval metric after install
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no

22%|██▏       | 286/1317 [00:39<02:30,  6.84it/s]


...

60%|██████    | 795/1317 [01:59<01:12,  7.18it/s]


98%|█████████▊| 50/51 [00:05<00:00,  8.36it/s]#033[A
#033[A
{'eval_loss': 0.2935323417186737, 'eval_precision': 0.6621699749470032, 'eval_recall': 0.616544051677732, 'eval_f1': 0.6385430217431703, 'eval_accuracy': 0.9107700005886855, 'eval_runtime': 6.9246, 'eval_samples_per_second': 469.341, 'eval_steps_per_second': 7.365, 'epoch': 2.0}
#015100%|██████████| 51/51 [00:06<00:00,  8.36it/s]#033[A
#033[A
75%|███████▌  | 989/1317 [02:34<00:47,  6.90it/s]


76%|███████▌  | 1000/1317 [02:36<00:48,  6.55it/s]
{'loss': 0.3056, 'learning_rate': 1.9400244798041617e-05, 'epoch': 2.28}
90%|████████▉ | 1179/1317 [03:04<00:21,  6.37it/s]


98%|█████████▊| 50/51 [00:05<00:00,  8.16it/s]#033[A
#033[A
{'eval_loss': 0.24818406999111176, 'eval_precision': 0.700082019502415, 'eval_recall': 0.6892158621927149, 'eval_f1': 0.6946064469460645, 'eval_accuracy': 0.9254135515394125, 'eval_runtime': 7.1323, 'eval_samples_per_second': 455.671, 'eval_steps_per_second': 7.151, 'epoch': 3.0}
#015100%|██████████| 51/51 [00:07<00:00,  8.16it/s]#033[A
#033[A
{'train_runtime': 212.842, 'train_samples_per_second': 197.907, 'train_steps_per_second': 6.188, 'train_loss': 0.46218288424527365, 'epoch': 3.0}
33%|███▎      | 17/51 [00:01<00:03,  8.87it/s]



2023-08-28 22:20:53 Uploading - Uploading generated training model35%|███▌      | 18/51 [00:01<00:03,  8.41it/s]
100%|██████████| 51/51 [00:07<00:00,  7.10it/s]
***** Eval results *****
2023-08-28 22:20:49,446 sagemaker-training-toolkit INFO     Waiting for the process to finish and give a return code.
2023-08-28 22:20:49,446 sagemaker-training-toolkit INFO     Done waiting for a return code. Received 0 from exiting process.
2023-08-28 22:20:49,447 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2023-08-28 22:22:39 Completed - Training job completed
Training seconds: 885
Billable seconds: 885
